# Improve C# Code Performance 

- [reference](https://medium.com/stackademic/stop-writing-slow-c-span-and-memory-are-the-upgrade-your-codebase-desperately-needs-d6d61240682b). 

In [1]:
using System;

var input = "123,45.67";
var parts = input.Split(','); //Split allocates memory for a new array, Each part becomes a new string in memory
//modern dotnet use splice with span to avoid allocations
var id = int.Parse(parts[0]);
var score = double.Parse(parts[1]);

Console.WriteLine($"ID: {id}, Score: {score}");

ID: 123, Score: 45.67


- Span<T> is a stack-only type that represents a contiguous region of arbitrary memory.  
- It can be used to work with slices of arrays, strings, or unmanaged memory without additional allocations.
- Using Span<T> can help improve performance by reducing memory allocations and garbage collection overhead, especially in high-performance scenarios.


In [2]:
{

    ReadOnlySpan<char> span = input.AsSpan(); // var 키워드 사용으로 컴파일러가 타입 추론
    var prefix = span.Split(",");
    foreach(var item in prefix)
    {
        Console.WriteLine(span[item].ToString());
    }
}

123
45.67


In [ ]:
//Span Enumerator Example

using System;
using System.Threading.Tasks;

public class Program
{
    private static readonly byte[] _array = new byte[5];

    public static void Main()
    {
        new Random(42).NextBytes(_array);
        Span<byte> span = _array;

        Task.Run( () => ClearContents() );

       EnumerateSpan(span);
    }

    public static void ClearContents()
    {
        Task.Delay(20).Wait();
        lock (_array)
        {
           Array.Clear(_array, 0, _array.Length);
        }
    }

    public static void EnumerateSpan(Span<byte> span)
    {
        lock (_array)
        {
            foreach (byte element in span) //Span<T>.Enumerator is a ref struct.
            {
                Console.WriteLine(element);
                Task.Delay(10).Wait();
            }
        }

    }
}
// The example displays output like the following:
//     62
//     23
//     186
//     0
//     0
Program.Main();

62
23
186
150
174


# ReadOnlySpan<T> 

- it cannot mutate the underlying data. It is perfect for reading strings, byte arrays, network buffers, and data you did not allocate.

# Span<T> 

- it's the mutable version. You use it when you want to write into memory. For example, formatting content into a buffer.

In [3]:
{
    var value = 12345;
    Span<char> buffer = stackalloc char[32];
    bool ok = value.TryFormat(buffer, out int written);
}

In [ ]:
//wrong buffer
{
    var buffer = new char[32]; //heap allocation
}

//No heap usage. No GC involvement. No object allocation.
{
    Span<char> buffer = stackalloc char[32];
}


In [4]:
using System;

public static class StackAllocExample
{
    public static void Main()
    {
        string original = "Hello, StackAlloc!";
        Console.WriteLine($"Original: {original}");
        
        string reversed = ReverseString(original);
        Console.WriteLine($"Reversed: {reversed}");
    }

    public static string ReverseString(string input)
    {
        if (string.IsNullOrEmpty(input)) return input;

        // 1. 힙(Heap)이 아닌 스택(Stack)에 메모리를 할당합니다.
        // new char[input.Length]를 쓰면 GC가 나중에 수거해야 하지만,
        // stackalloc은 메서드가 끝나면 즉시 자동 소멸됩니다.
        Span<char> buffer = stackalloc char[input.Length];

        // 2. 데이터를 뒤집어서 스택 버퍼에 저장합니다.
        for (int i = 0; i < input.Length; i++)
        {
            buffer[i] = input[input.Length - 1 - i];
        }

        // 3. 최종 결과만 문자열(힙)로 변환하여 반환합니다.
        // 중간 연산 과정에서는 힙 할당이 전혀 발생하지 않았습니다.
        return buffer.ToString();
    }
}
StackAllocExample.Main();

Original: Hello, StackAlloc!
Reversed: !collAkcatS ,olleH


# stackalloc을 활용한 고성능 메모리 할당 예제
`stackalloc`은 힙(Heap)이 아닌 **스택(Stack)** 메모리에 배열을 할당하는 키워드입니다. 이를 `Span<T>`과 함께 사용하면 가비지 컬렉터(GC)의 관리 없이 매우 빠르게 임시 버퍼를 생성하고 사용할 수 있습니다. 주로 짧은 시간 동안 유지되는 데이터 처리에 사용됩니다.

## 예시 파일
[Microsoft 공식 stackalloc 예제 코드](https://github.com/dotnet/samples/blob/main/snippets/csharp/language-reference/operators/Stackalloc/Program.cs)

`stackalloc`을 사용하면 `new` 키워드를 사용할 때 발생하는 힙 메모리 할당과 나중에 GC가 이를 청소하는 비용을 절약할 수 있습니다.

아래는 문자열을 뒤집는(Reverse) 기능을 구현할 때, 중간 임시 저장소를 힙 배열(`char[]`) 대신 `stackalloc`을 사용하여 최적화한 예제입니다.

### 핵심 포인트
1.  **속도 향상**: 스택 할당은 CPU 명령 몇 개로 끝나므로 힙 할당보다 훨씬 빠릅니다.
2.  **GC 부담 없음**: 메서드(`ReverseString`)가 종료되면 스택 메모리는 자동으로 정리되므로 GC가 관여하지 않습니다.
3.  **Span과의 결합**: 과거에는 `unsafe` 키워드와 포인터를 써야 했지만, 현재는 `Span<T>` 덕분에 안전하게(Safe context) 사용할 수 있습니다.
4.  **주의사항**: 스택 메모리는 크기가 작습니다(보통 1MB). 너무 큰 배열(예: 수만 개 이상의 요소)을 `stackalloc`으로 할당하면 **StackOverflowException**이 발생하여 프로그램이 종료될 수 있습니다. 따라서 작은 크기의 버퍼에만 사용해야 합니다.

### 추가 자료
- [stackalloc 식(C# 참조)](https://learn.microsoft.com/ko-kr/dotnet/csharp/language-reference/operators/stackalloc)
- [Span<T> 및 메모리 효율성 가이드](https://learn.microsoft.com/ko-kr/dotnet/standard/memory-and-spans/memory-and-spans-usage-guidelines)

In [ ]:
using System;
using System.IO;
using System.Threading.Tasks;

public class Processor
{
    private Memory<byte> _buffer;

    public Processor(int size)
    {
        _buffer = new byte[size];
    }

    public async Task FillAsync(Stream source)
    {
        int read = await source.ReadAsync(_buffer);
        Span<byte> span = _buffer.Span.Slice(0, read);
        Process(span);
    }
    private void Process(Span<byte> data)
    {
        Console.WriteLine($"Processing {data.Length} bytes...");

        long sum = 0;
        
        // Span을 순회하며 데이터 처리 (배열처럼 인덱스 접근 가능)
        foreach (byte b in data)
        {
            sum += b;
        }

        Console.WriteLine($"Data Sum: {sum}");

        // 예시: 첫 번째 바이트 값 확인
        if (data.Length > 0)
        {
            Console.WriteLine($"First Byte: {data[0]}");
        }
    }
}

## 핵심 설명
- Span<byte> 매개변수: Process 메서드는 Memory<byte>가 아닌 Span<byte>를 받습니다. 이는 _buffer.Span 프로퍼티를 통해 매우 저렴하게 생성됩니다.
- 동기 메서드: Span<T>는 ref struct이기 때문에 힙에 저장될 수 없습니다. 따라서 async/await 메서드의 매개변수나 지역 변수로 사용할 수 없어, Process는 반드시 동기 메서드(void 등)여야 합니다.
- 효율성: Slice(0, read)를 호출해도 새로운 배열이 생성되지 않고, 기존 메모리의 특정 영역을 가리키는 뷰(View)만 생성되므로 성능이 매우 뛰어납니다.

# Memory<T>와 Span<T>의 차이 및 사용 패턴
`Span<T>`은 **스택(Stack)**에만 존재할 수 있어 빠르지만 제약이 많은 반면, `Memory<T>`는 **힙(Heap)**에 저장될 수 있어 비동기 작업이나 클래스 필드로 사용할 수 있습니다. `Memory<T>`로 데이터를 저장/전달하고, 실제 데이터를 처리할 때는 `Span`으로 변환하여 고성능으로 작업하는 것이 표준 패턴입니다.

## 예시 파일
[Memory<T>와 Span<T> 사용 가이드 (Microsoft Docs)](https://learn.microsoft.com/ko-kr/dotnet/standard/memory-and-spans/memory-and-spans-usage-guidelines)

## 답변

### 1. 차이점 핵심 요약

| 특성 | Span&lt;T&gt; | Memory&lt;T&gt; |
| :--- | :--- | :--- |
| **메모리 위치** | **스택(Stack) 전용** (`ref struct`) | 스택 또는 **힙(Heap)** 모두 가능 |
| **클래스 필드** | 불가능 (힙에 저장 불가) | **가능** (클래스 멤버로 저장 가능) |
| **비동기(async)** | `await` 전후로 유지 불가 | **가능** (`await` 넘어서 유지 가능) |
| **용도** | 데이터를 **처리(Processing)**할 때 | 데이터를 **저장(Storage)**하거나 전달할 때 |
| **성능** | 매우 빠름 (인덱싱 최적화) | Span보다 약간 느림 (변환 필요) |

### 2. 왜 Memory 선언 후 Span을 호출하는가?

`Memory<T>`를 선언해두고 굳이 `.Span` 프로퍼티를 호출해서 `Span<T>`으로 변환해 사용하는 이유는 **"역할의 분리"**와 **"성능"** 때문입니다.

1.  **저장과 이동 (Memory의 역할)**:
    *   비동기 메서드(`async/await`)나 클래스의 필드(`private Memory<byte> _buffer`)처럼 데이터가 힙에 오랫동안 살아있어야 하는 경우 `Memory<T>`에 담아둡니다. `Span`은 여기에 존재할 수 없기 때문입니다.

2.  **고속 처리 (Span의 역할)**:
    *   실제로 데이터를 읽거나 쓸 때(Loop 돌기, 파싱 등)는 `Span`이 훨씬 빠릅니다.
    *   `Memory<T>` 자체의 인덱서보다 `Span`으로 변환한 뒤 접근하는 것이 CPU 차원에서 최적화되어 있습니다.
    *   따라서 **"저장은 Memory에, 처리는 Span으로"** 하는 패턴을 사용합니다.

### 3. 코드 예시

```csharp
using System;
using System.Threading.Tasks;

public class AsyncDataHandler
{
    // 1. 클래스 필드에는 Span을 쓸 수 없으므로 Memory<byte>를 사용해 저장합니다.
    private Memory<byte> _storage;

    public AsyncDataHandler(int size)
    {
        _storage = new byte[size];
    }

    // 2. 비동기 메서드에서는 Memory<T>를 매개변수로 받거나 사용합니다.
    public async Task LoadAndProcessAsync()
    {
        // 비동기 작업: 여기서는 Memory<T> 상태로 전달해야 합니다.
        // (Span은 await를 건너뛸 수 없음)
        await SimulateNetworkReadAsync(_storage);

        // 3. 데이터 처리가 시작되는 시점에 .Span을 호출합니다.
        // 이 시점부터는 동기적으로 동작하며, Span의 고성능 이점을 활용합니다.
        Span<byte> spanView = _storage.Span;
        
        // 실제 데이터 조작은 Span으로 수행
        for (int i = 0; i < spanView.Length; i++)
        {
            spanView[i] = (byte)(spanView[i] * 2);
        }
    }

    private async Task SimulateNetworkReadAsync(Memory<byte> buffer)
    {
        await Task.Delay(100); // IO 시뮬레이션
        // ... 데이터 채우기 ...
    }
}
```

### 요약
- **Memory<T>**: 데이터를 **가방**에 넣어두는 것과 같습니다. 가방은 선반(Heap)에 올릴 수도 있고 친구에게 택배(Async)로 보낼 수도 있습니다.
- **Span<T>**: 가방에서 물건을 꺼내 **손**에 쥐고 작업하는 것과 같습니다. 손에 쥐고 있어야(Stack) 가장 빠르게 작업할 수 있지만, 손에 쥔 채로 잠을 자거나(await) 멀리 이동할 수는 없습니다.
- **패턴**: 가방(`Memory`)에 담아 이동하다가, 작업할 때만 손(`Span`)에 쥐고 처리합니다.

### 추가 자료
- [Memory&lt;T&gt; 및 Span&lt;T&gt; 사용 지침](https://learn.microsoft.com/ko-kr/dotnet/standard/memory-and-spans/memory-and-spans-usage-guidelines)
- [C#의 모든 Span에 대한 설명 (Microsoft Blog)](https://devblogs.microsoft.com/dotnet/all-about-span-part-1-introduction/)

# 숨겨진 메모리 할당 방지 및 최적화
Span 기반 프로그래밍의 가장 큰 장점은 평범해 보이는 코드가 얼마나 많은 불필요한 메모리(Garbage)를 생성하는지 깨닫게 해준다는 점입니다. 다음은 흔히 발생하지만 눈에 띄지 않게 메모리를 할당하는 패턴들과 이를 최적화하는 방법입니다.

## 예시 파일
[C# 고성능 코딩 패턴 예제 (Microsoft Docs)](https://learn.microsoft.com/ko-kr/dotnet/standard/memory-and-spans/memory-and-spans-usage-guidelines)

## 답변

### 1. 배열이나 문자열에 LINQ 사용 (Using LINQ on arrays or strings)
*   **문제점**: `Select`, `Where` 같은 LINQ 메서드는 내부적으로 열거자(Enumerator), 델리게이트, 그리고 종종 새로운 배열을 힙에 할당합니다.
*   **해결**: `Span<T>` 위에서 `foreach`나 `for` 루프를 사용하거나, `MemoryExtensions`에 있는 Span 전용 메서드를 사용하세요.

### 2. 루프 내에서 string.Split 사용 (Using string.Split in loops)
*   **문제점**: `Split`은 호출할 때마다 문자열 배열(`string[]`)과 잘라진 각각의 새 문자열 객체들을 힙에 생성합니다.
*   **해결**: `Span<T>.Split`을 사용하세요. 이는 문자열을 생성하지 않고 원본의 인덱스 범위(Range)만 반환합니다.

### 3. 반복적인 Substring 사용 (Using Substring repeatedly)
*   **문제점**: `Substring`은 원본 문자열의 일부를 복사하여 **새로운 문자열 객체**를 만듭니다.
*   **해결**: `Slice`를 사용하세요. 메모리 복사 없이 기존 메모리를 가리키는 뷰(View)만 생성합니다.

### 4. 루프 내 문자열 연결 (Concatenating strings inside loops)
*   **문제점**: 문자열은 불변(Immutable)입니다. `str += "a"`를 할 때마다 이전 문자열을 버리고 더 큰 새 문자열을 할당합니다.
*   **해결**: `StringBuilder`를 사용하거나, 크기를 아는 경우 `Span<char>` 버퍼에 직접 쓰세요.

### 5. 작은 세그먼트에 Encoding.GetString 사용 (Using Encoding.GetString on small segments)
*   **문제점**: 바이트 배열의 일부를 문자열로 변환할 때마다 새 객체가 생성됩니다.
*   **해결**: 파싱이 목적이라면 문자열로 변환하지 말고 `Span<byte>` 상태에서 직접 처리하거나 `Encoding.GetChars`를 통해 스택 버퍼에 문자를 쓰세요.

### 6. 슬라이스만 필요한 경우의 BitConverter 사용 (Using BitConverter on arrays)
*   **문제점**: `BitConverter.ToInt32` 등을 사용하기 위해 배열을 자르거나(`Skip/Take`) 복사하면 할당이 발생합니다.
*   **해결**: `BinaryPrimitives` 클래스를 사용하여 `Span<byte>`에서 직접 값을 읽으세요.

### 7. 단순 파싱에 정규식 사용 (Using regular expressions for simple parsing)
*   **문제점**: 정규식 엔진은 초기화 비용이 들고 `Match` 객체 등을 생성합니다.
*   **해결**: 복잡한 패턴이 아니라면 `IndexOf`, `Slice`, `StartsWith` 등을 조합해 직접 파싱하는 것이 훨씬 빠르고 할당이 없습니다.

### 코드 비교 예제 (Substring vs Slice)

```csharp
using System;
using System.Buffers.Binary;

public class AllocationExample
{
    public void BadPattern(string input)
    {
        // [Bad] Substring은 힙에 새로운 문자열 "Hello"를 할당함
        string sub = input.Substring(0, 5); 
        
        // [Bad] LINQ는 숨겨진 할당(Enumerator 등)을 유발함
        int count = input.Count(c => c == 'o');
    }

    public void GoodPattern(string input)
    {
        ReadOnlySpan<char> span = input.AsSpan();

        // [Good] Slice는 할당 없이 원본의 0~5 위치만 가리킴 (0 할당)
        ReadOnlySpan<char> slice = span.Slice(0, 5);

        // [Good] Span 기반 루프 (0 할당)
        int count = 0;
        foreach (char c in span)
        {
            if (c == 'o') count++;
        }
    }
    
    public void BitConverterExample(ReadOnlySpan<byte> data)
    {
        // [Good] 배열 복사 없이 Span에서 직접 정수 읽기
        int value = BinaryPrimitives.ReadInt32LittleEndian(data);
    }
}
```

### 추가 자료
- [BinaryPrimitives 클래스 (고성능 바이너리 처리)](https://learn.microsoft.com/ko-kr/dotnet/api/system.buffers.binary.binaryprimitives)
- [MemoryExtensions 클래스 (Span용 확장 메서드)](https://learn.microsoft.com/ko-kr/dotnet/api/system.memoryextensions)

# BinaryPrimitives와 MemoryExtensions 활용 예제
`BinaryPrimitives`는 바이트 스팬(`Span<byte>`)에서 데이터를 엔디안(Endianness)에 맞춰 효율적으로 읽고 쓰는 기능을 제공하며, `MemoryExtensions`는 `Span<T>`에 대해 검색, 비교, 공백 제거(Trim) 등의 기능을 메모리 할당 없이 수행할 수 있게 해줍니다. 이 두 클래스는 고성능 C# 프로그래밍의 핵심 도구입니다.

## 예시 파일
[BinaryPrimitives 공식 문서 및 예제](https://learn.microsoft.com/ko-kr/dotnet/api/system.buffers.binary.binaryprimitives)

## 답변

### 1. BinaryPrimitives 예제
`BitConverter`를 사용하면 시스템의 엔디안에 의존하거나, 부분 배열을 만들기 위해 메모리를 할당해야 할 때가 많습니다. `BinaryPrimitives`는 `Span<byte>`를 사용하여 할당 없이 명시적인 엔디안으로 데이터를 처리합니다.



In [5]:
using System;
using System.Buffers.Binary;

public class BinaryProcessing
{
    public static void Run()
    {
        // 시나리오: 네트워크 패킷 헤더 (4바이트 ID + 2바이트 길이)
        // 빅 엔디안(Big Endian)으로 들어온 데이터를 가정
        byte[] packetData = { 0x00, 0x00, 0x04, 0xD2, 0x00, 0x10 }; // ID: 1234, Length: 16
        
        ReadOnlySpan<byte> span = packetData;

        // 1. 읽기 (Read)
        // BitConverter와 달리 Span을 직접 받아 처리하며, 엔디안을 명시할 수 있습니다.
        int id = BinaryPrimitives.ReadInt32BigEndian(span.Slice(0, 4));
        short length = BinaryPrimitives.ReadInt16BigEndian(span.Slice(4, 2));

        Console.WriteLine($"Packet ID: {id}, Length: {length}");

        // 2. 쓰기 (Write)
        Span<byte> buffer = stackalloc byte[6];
        
        // 스택 버퍼에 리틀 엔디안(Little Endian)으로 기록
        BinaryPrimitives.WriteInt32LittleEndian(buffer.Slice(0, 4), id);
        BinaryPrimitives.WriteInt16LittleEndian(buffer.Slice(4, 2), length);

        Console.Write("Written (Little Endian): ");
        foreach (var b in buffer) Console.Write($"{b:X2} ");
        Console.WriteLine();
    }
}


### 2. MemoryExtensions 예제
`string` 클래스의 메서드(`Trim`, `IndexOf`, `StartsWith` 등)와 유사하지만, 새로운 문자열을 생성하지 않고 `Span`의 범위를 조정하거나 결과를 반환하는 고성능 메서드들입니다.


In [6]:
using System;

public class StringOptimization
{
    public static void Run()
    {
        // 시나리오: 로그 메시지 파싱 "  [INFO] User:Donghun  "
        string logEntry = "  [INFO] User:Donghun  ";
        ReadOnlySpan<char> span = logEntry.AsSpan();

        // 1. Trim (공백 제거)
        // 새로운 문자열을 만들지 않고, 공백이 제외된 범위를 가리키는 새 Span을 반환합니다.
        ReadOnlySpan<char> trimmed = span.Trim(); 

        // 2. StartsWith (시작 문자열 확인)
        if (trimmed.StartsWith("[INFO]"))
        {
            // 3. IndexOf & Slice (특정 위치 찾기 및 자르기)
            int colonIndex = trimmed.IndexOf(':');
            if (colonIndex != -1)
            {
                // ':' 다음부터 끝까지 자름
                ReadOnlySpan<char> username = trimmed.Slice(colonIndex + 1);
                
                // 4. SequenceEqual (문자열 비교)
                // username.ToString() == "Donghun" 처럼 변환하지 않고 비교
                bool isTargetUser = username.SequenceEqual("Donghun");

                Console.WriteLine($"User found: {username.ToString()} (Match: {isTargetUser})");
            }
        }
        
        // 5. ToUpperInvariant (대소문자 변환)
        // 원본은 불변이므로, 변환 결과를 담을 버퍼가 필요합니다.
        Span<char> upperBuffer = stackalloc char[trimmed.Length];
        trimmed.ToUpperInvariant(upperBuffer);
        
        Console.WriteLine($"Upper: {upperBuffer.ToString()}");
    }
}

### 핵심 요약
1.  **BinaryPrimitives**: `byte[]`나 `Span<byte>`에서 `int`, `long` 등의 기본 타입을 **할당 없이**, **엔디안을 지정**하여 읽고 쓸 때 사용합니다.
2.  **MemoryExtensions**: `Span<T>`에서 `Trim`, `IndexOf`, `CompareTo` 등 배열/문자열 조작 기능을 **메모리 복사 없이** 수행할 때 사용합니다.

### 추가 자료
- [BinaryPrimitives 클래스 (Microsoft Docs)](https://learn.microsoft.com/ko-kr/dotnet/api/system.buffers.binary.binaryprimitives)
- [MemoryExtensions 클래스 (Microsoft Docs)](https://learn.microsoft.com/ko-kr/dotnet/api/system.memoryextensions)

# Span과 Memory 최적화가 중요한 이유와 결론
단순히 코드가 "돌아가는 것"을 넘어, .NET 런타임과 조화를 이루며 효율적으로 동작하는 고성능 코드를 작성해야 하는 이유와 그 구체적인 실천 방법에 대한 번역 및 설명입니다.

## 예시 파일
[Span<T> 및 Memory<T> 사용 가이드 (Microsoft Docs)](https://learn.microsoft.com/ko-kr/dotnet/standard/memory-and-spans/memory-and-spans-usage-guidelines)

## 답변

### 1. 원문 번역

#### 이것이 생각보다 훨씬 중요한 이유 (Why All This Matters More Than You Think)
일부 개발자들은 성능 최적화가 게임이나 대규모 백엔드 서비스에서만 중요하다고 생각합니다. 하지만 현실은 간단합니다. 여러분의 코드는 **런타임(Runtime)과 협력하거나, 아니면 런타임과 싸우게 됩니다.**

여러분이 메모리 할당을 피할 때마다 가비지 컬렉터(GC)가 프로그램을 멈추는 시간(Pause)이 줄어듭니다. 문자열 복사를 피할 때마다 메모리 대역폭이 보존됩니다. `stackalloc`을 사용하는 모든 곳에서 힙(Heap) 메모리에 대한 부담이 완전히 사라집니다. 코드가 모바일 기기나 저사양 하드웨어에서 실행될 때, 이러한 영향은 더욱 눈에 띄게 나타납니다.

Span을 사용하면 코드 전반에 걸쳐 다음과 같은 성능 향상 효과를 얻을 수 있습니다:
*   메모리 할당 감소
*   GC 주기(Cycle) 감소
*   더 빠른 파싱(Parsing)
*   더 빠른 포맷팅(Formatting)
*   더 빠른 메모리 접근
*   명시적으로 요청하지 않는 한 복사(Copying) 없음

가장 좋은 점은 코드가 더 **의도적(Intentional)**으로 변한다는 것입니다. Span 기반 API를 사용하면 데이터가 시스템 내에서 어떻게 이동하는지 더 잘 인식하게 됩니다. 이는 데이터를 '할당하고 복제'하는 관점이 아니라, **'자르고(Slicing) 들여다보는(Viewing)'** 관점에서 생각하도록 가르쳐줍니다.

#### 결론 (Conclusion)
애플리케이션 전체를 당장 Span으로 다시 작성할 필요는 없습니다. **작게 시작하세요.**
*   `Substring`을 `AsSpan`으로 바꾸세요.
*   `Split`을 Span 기반 파서로 바꾸세요.
*   임시 배열을 `stackalloc`으로 바꾸세요.
*   오래 유지되는 바이트 버퍼를 `Memory<T>`로 바꾸세요.

시간이 지나면 이러한 패턴이 자연스러워질 것입니다. 여러분은 설계 단계부터 더 빠른 코드를 작성하게 될 것입니다.

이 주제는 경험 많은 개발자와 무의식적으로 GC에 의존하는 개발자를 구분 짓는 C#의 중요한 주제 중 하나입니다. 최신 .NET은 안전하고(Safe), 표현력이 풍부하며(Expressive), 가독성이 좋으면서도(Readable) 로우 레벨(Low-level) 느낌의 코드를 작성할 수 있는 도구를 제공합니다.

---

### 2. 상세 설명 및 초보자를 위한 가이드

이 글은 단순히 "Span이 빠르다"는 사실을 넘어, **프로그래머의 사고방식(Mindset)** 변화를 강조하고 있습니다.

#### 1) "런타임과 싸운다"는 의미
C#은 가비지 컬렉터(GC)가 메모리를 관리해주는 언어입니다.
*   **싸우는 코드**: 개발자가 `new`를 남발하여 쓰레기(Garbage)를 계속 만들면, GC는 바빠집니다. GC가 청소를 시작하면 실행 중인 프로그램이 순간적으로 멈춥니다(Stop-the-world). 이는 런타임에 부담을 주는 행위입니다.
*   **협력하는 코드**: `Span`을 사용하여 불필요한 메모리를 만들지 않으면, GC는 할 일이 없어집니다. 프로그램은 멈춤 없이 부드럽게 실행됩니다.

#### 2) "의도적(Intentional)"인 코드란?
기존 방식(`Substring`)은 원본이 1GB짜리 문자열이라도, 5글자를 자르기 위해 새로운 메모리 공간을 할당하고 복사했습니다. 개발자는 이 비용을 잊기 쉽습니다.
하지만 `Span`을 쓰면 **"나는 복사본을 만드는 게 아니라, 원본의 이 부분만 빌려서 보겠다"**는 의도가 코드에 명확히 드러납니다. 이를 **뷰(View)**라고 합니다.

#### 3) 실천 가이드 (Start Small)
초보자가 당장 적용할 수 있는 가장 쉬운 패턴은 문자열 처리입니다.

**[기존 방식 - 메모리 할당 발생]**
```csharp
string date = "2023-10-25";
string year = date.Substring(0, 4); // "2023"이라는 새 문자열 객체 생성 (힙 할당)
int yearNum = int.Parse(year);
```

**[Span 방식 - 메모리 할당 0]**
```csharp
ReadOnlySpan<char> dateSpan = "2023-10-25".AsSpan();
ReadOnlySpan<char> yearSpan = dateSpan.Slice(0, 4); // 메모리 복사 없이 위치만 가리킴
int yearNum = int.Parse(yearSpan); // Span을 바로 파싱
```

### 추가 자료
- [가비지 수집(GC)의 기본 사항](https://learn.microsoft.com/ko-kr/dotnet/standard/garbage-collection/fundamentals)
- [.NET의 Span<T> 소개 (Microsoft DevBlogs)](https://devblogs.microsoft.com/dotnet/all-about-span-part-1-introduction/)